# Final Project "Прогнозирование влияния химических показателей состава вина на его качество"

В своем исследовании, я хочу продемонстрировать то, какие показатели влияют на качество красного вина Vinho Verde.
Хочется отметить, что в данной модели не хватает некоторых иных показателей для максимально применимой на практике модели, но все показатели касательно химического состава, могут быть использованы производителями для увеличения вероятности попадания их вина в самые высокие рейтинги, поскольку оценка субъективна.
Сначала, я посмотрю основные данные и построю базовую регрессионную модель, а дальше еще три модели и одну классификацию.

In [ ]:
#импортирую библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
df = pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
#имя столбцов
column = df.columns
names = column[:-1]

In [ ]:
#загружаю данные с 11 признаками/качествами
features = np.array(df[column[:-1]])
#ну и отдельно оценку вина = целевую функцию
target = np.array(df[column[-1]])
#и делю на тренировочные и тестовые данные
features_train, features_test, target_train,target_test = train_test_split(features,target,test_size=0.1,random_state=1)


In [ ]:
#объект базовой регрессионной модели
dummy_regression = DummyRegressor(strategy='mean')
#тренировка базовой модели
model = dummy_regression.fit(features_train,target_train)
#прогноз целевой функции на тестовых признаках
target_predicted = model.predict(features_test)

In [ ]:
#сделала оценку коэф-том детерминации
dummy_r2 = model.score(features_test,target_test)
print('Оценка коэффициента детерминации:',dummy_r2)


Вышла плохая корреляция.

### Далее, попробую сделать линейную регрессиию, чтобы натренировать модель:

In [ ]:
#строю график
def plot(regression_coef):
    plt.title("модельные коэффициенты")
    plt.bar(range(features.shape[1]),regression_coef)
    plt.xticks(range(features.shape[1]),names,rotation=90)
    plt.show()

In [ ]:
#создаю объект линейной регресии
lin_regression = LinearRegression()
#подгонка линейной регрессии
model = lin_regression.fit(features_train,target_train)
#прогноз целевой функции на тестовых признаках
target_predicted = model.predict(features_test)
#оценю модель коэф-ом детерминации
lin_r2 = model.score(features_test,target_test)
print('Оценка коэффициента  детерминации:',lin_r2)

In [ ]:
#эффект единичного изменения на вектор целей, модельные коэффициенты
lin_regression_coef = model.coef_
plot(lin_regression_coef)

Из графика исходит то, что увеличение плотности как следствие максимально негативно сказывается на целевой функции (качестве вина) 

### Логистическая регрессия:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
#оформляю матрицу ошибок
def matrix_plot():
    matrix = confusion_matrix(target_test,target_predicted)
    matrix_name = sorted(set(target_predicted) | set(target_test)) #создание осей для х и у - это оценки
    df = pd.DataFrame(matrix,index=matrix_name,columns=matrix_name)
    sns.heatmap(df,annot=True,cbar=None,cmap="Blues")
    plt.title("Матрица ошибок")
    plt.ylabel("Истинный класс")
    plt.xlabel("Предсказанный класс")
    plt.show()

In [ ]:
log_regression = LogisticRegression(random_state=0,class_weight="balanced")
model = log_regression.fit(features_train,target_train)
target_predicted = model.predict(features_test) #прогноз целевой функции на тестовых признаках
log_r2 = model.score(features_test,target_test) #оценка модели коэф-ом детерминации
print('Оценка коэффициента  детерминации:',log_r2)
matrix_plot()

На матрице можно заметить небольшое отклонение от диагонали, небольшое количество ошибок классификации

### Тренировка регрессивного дерева принятия решений:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
decisiontree_regression = DecisionTreeRegressor(random_state=0) 
model = decisiontree_regression.fit(features_train,target_train)
target_predicted = model.predict(features_test)
decisiontree_r2 = model.score(features_test,target_test)
print('Оценка коэффициента  детерминации:',decisiontree_r2)
matrix_plot()

Не смотря на небольшое отклонение от диагонали, R2 очень низкий, нужно разбираться в данных, модель не применима к данной задаче.

### Тренировка регрессивного случайного леса:

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
def importance(model_x):
    importances = model_x.feature_importances_
    #строю график важность признаков
    indices = np.argsort(importances)[::-1] #сортировка в нисходящем порядке
    names_in_plot = [names[i] for i in indices] #потом перераспределяю имена
    plt.title("Важность признаков") 
    plt.bar(range(features.shape[1]),importances[indices])
    plt.xticks(range(features.shape[1]),names_in_plot,rotation=90)
    plt.show()

In [ ]:
randomforest_regression = RandomForestRegressor(random_state=0,n_jobs=-1) #объект 
model = randomforest_regression.fit(features_train,target_train)
randomforest_regression = model.predict(features_test)
randomforest_r2 = model.score(features_test,target_test)
print('Оценка коэффициента  детерминации:',randomforest_r2)
importance(model)

Со средней оценкой, приходим к выводу диаграммы, проранжированной по степени важности признаков для определения качества вина. На первом месте стоит показатель Алкоголя, что дает понять, то, что выставления субъективных оценок, эксперты, в первую очередь обращали на градусность напитка.

### Было решено попробовать построить еще и классификацию! Потому что в разных видах регрессии получаются разные признаки.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.neighbors import RadiusNeighborsClassifier,KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import learning_curve

In [ ]:
def classifier(method):
    model = method.fit(features_train,target_train)
    importance(model)
    print('Точность предсказаний трен', accuracy_score(target_train,model.predict(features_train)))
    print('Точность предсказаний тест', accuracy_score(target_test,model.predict(features_test)))
    train_sizes, train_scores, test_scores = learning_curve (method,features_train,target_train,cv=10,scoring='accuracy', n_jobs=-1, train_sizes=np.linspace(0.01,1.0,50))
    train_mean = np.mean(train_scores,axis=1)
    test_std = np.std(test_scores,axis=1)
    test_mean = np.mean(test_scores,axis=1)
    plt.plot(train_sizes,train_mean,'--',color="#111111",label="Тренировочная оценка")
    plt.plot(train_sizes,test_mean,color="#111111",label="Перекрестно-проверочная оценка")
    plt.fill_between(train_sizes,test_mean-test_std,test_mean+test_std,color="#DDDDDD")
    plt.title("Кривая заучивания")
    plt.xlabel("Размер тренировочного набора")
    plt.ylabel("Оценка точности")
    plt.grid()
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
decisiontree = DecisionTreeClassifier(random_state=0)
randomforest = RandomForestClassifier(random_state=0,n_jobs=-1)
ABS = AdaBoostClassifier(random_state=0)
gaussonnb = GaussianNB()

Использую классифаер по дереву решений, можно изменить на любую другую из перечисленных выше:

In [ ]:
classifier(decisiontree)

In [ ]:
classifier(randomforest)

In [ ]:
classifier(ABS)

### Исходя из проведенного анализа, можно сказать, что степень влияния алкоголя на качество вина в большей степени доказана классификацией по дереву принятия решений и методом тренировки регрессивного случайного леса, они и являются наиболее подходящими моделями для классификации.

*learning curve - кривая заучивания (по Элбону) - графическое представление того, как меняется точность в зависимости от размера обучающей выборки 

Для выполнения работы была использована книга Ричарта В. и Коэльо П.Л. "Построние систем машинного обучения на языке Python", К. Элбона "Машинное обучение с использованием Python. Сборник рецептов" и статьи с интернет-портала habr.ru. (например, статья https://habr.com/ru/company/mlclass/blog/247751/)